In [4]:
import sqlite3
import pandas as pd
import os

folder_path = r"C:\After Backup\Virtual Internship\Week 6\2019"

sqlite_files = [file for file in os.listdir(folder_path) if file.endswith(".sqlite")]
print("SQLite files found:", sqlite_files)

SQLite files found: ['2019-01.sqlite', '2019-02.sqlite', '2019-03.sqlite', '2019-04.sqlite', '2019-05.sqlite', '2019-06.sqlite', '2019-07.sqlite', '2019-08.sqlite', '2019-09.sqlite', '2019-10.sqlite', '2019-11.sqlite', '2019-12.sqlite']


In [2]:
sample_db_path = os.path.join(folder_path, '2019-01.sqlite')

conn = sqlite3.connect(sample_db_path)

query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)

print("Tables in 2019-01.sqlite:")
print(tables)

Tables in 2019-01.sqlite:
       name
0  tripdata


In [3]:
for file in sqlite_files:
    db_path = os.path.join(folder_path, file)
    csv_name = file.replace(".sqlite", ".csv")
    csv_path = os.path.join(folder_path, csv_name)

    if os.path.exists(csv_path):
        print(f"Already exists: {csv_name} — skipping")
        continue

    print(f"Converting {file} to {csv_name}")
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT * FROM tripdata", conn)
    df.to_csv(csv_path, index=False)
    conn.close()

print("All SQLite files converted to CSVs.")

Converting 2019-01.sqlite to 2019-01.csv
Converting 2019-02.sqlite to 2019-02.csv
Converting 2019-03.sqlite to 2019-03.csv
Converting 2019-04.sqlite to 2019-04.csv
Converting 2019-05.sqlite to 2019-05.csv
Converting 2019-06.sqlite to 2019-06.csv
Converting 2019-07.sqlite to 2019-07.csv
Converting 2019-08.sqlite to 2019-08.csv
Converting 2019-09.sqlite to 2019-09.csv
Converting 2019-10.sqlite to 2019-10.csv
Converting 2019-11.sqlite to 2019-11.csv
Converting 2019-12.sqlite to 2019-12.csv
All SQLite files converted to CSVs.


In [4]:
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]
csv_files.sort()
print("Monthly CSVs found:", csv_files)

Monthly CSVs found: ['2019-01.csv', '2019-02.csv', '2019-03.csv', '2019-04.csv', '2019-05.csv', '2019-06.csv', '2019-07.csv', '2019-08.csv', '2019-09.csv', '2019-10.csv', '2019-11.csv', '2019-12.csv']


In [3]:
import os
import pandas as pd

folder_path = r"C:\After Backup\Virtual Internship\Week 6\2019"

In [2]:
csv_files = [file for file in os.listdir(folder_path) if file.endswith(".csv")]
csv_files.sort()
print("CSV files:", csv_files)

CSV files: ['2019-01.csv', '2019-02.csv', '2019-03.csv', '2019-04.csv', '2019-05.csv', '2019-06.csv', '2019-07.csv', '2019-08.csv', '2019-09.csv', '2019-10.csv', '2019-11.csv', '2019-12.csv', 'combined_2019.csv']


In [4]:
csv_files = sorted([file for file in os.listdir(folder_path) if file.endswith(".csv")])
csv_files = csv_files[:3]  # Only keep the first 3 files to keep it >2GB but avoid system overloading
print("CSV files to combine:", csv_files)

CSV files to combine: ['2019-01.csv', '2019-02.csv', '2019-03.csv']


In [5]:
first_file = True
combined_csv_path = os.path.join(folder_path, "combined_3_months.csv")

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    print(f"Processing {file}")

    try:
        df = pd.read_csv(file_path, low_memory=False, on_bad_lines='skip')
        df.to_csv(combined_csv_path, mode='w' if first_file else 'a',
                  index=False, header=first_file)
        first_file = False
    except Exception as e:
        print(f"Failed to process {file}: {e}")

Processing 2019-01.csv
Processing 2019-02.csv
Processing 2019-03.csv


In [6]:
combined_file_path = os.path.join(folder_path, "combined_3_months.csv")
size_in_gb = os.path.getsize(combined_file_path) / (1024**3)

print(f"File size: {size_in_gb:.2f} GB")

File size: 2.58 GB


In [7]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Time taken: {end - start:.2f} seconds")
        return result
    return wrapper

In [8]:
@timer
def read_with_pandas():
    import pandas as pd
    df = pd.read_csv(combined_file_path)
    print("Pandas: Loaded", df.shape[0], "rows and", df.shape[1], "columns")
    return df

df_pandas = read_with_pandas()

Pandas: Loaded 22519712 rows and 18 columns
Time taken: 176.01 seconds


In [9]:
@timer
def read_with_dask():
    import dask.dataframe as dd
    df = dd.read_csv(combined_file_path)
    print("Dask: DataFrame created (delayed loading)")
    print("Shape (approx):", df.shape)
    return df

df_dask = read_with_dask()

Dask: DataFrame created (delayed loading)
Shape (approx): (<dask_expr.expr.Scalar: expr=ArrowStringConversion(frame=FromMapProjectable(e891dea)).size() // 18, dtype=int64>, 18)
Time taken: 9.55 seconds


**Modin could not be benchmarked due to environment-specific issues with Ray on Jupyter under Windows.**

In [3]:
import pandas as pd
import os

folder_path = r"C:\After Backup\Virtual Internship\Week 6\2019"
combined_file_path = os.path.join(folder_path, "combined_3_months.csv")

df_pandas = pd.read_csv(combined_file_path, nrows=1000, low_memory=False, on_bad_lines='skip')

print("Sample loaded:", df_pandas.shape)

Sample loaded: (1000, 18)


In [4]:
print("Before cleaning:", df_pandas.columns.tolist())

df_pandas.columns = df_pandas.columns.str.strip().str.replace('[^A-Za-z0-9_]+', '', regex=True)

print("After cleaning:", df_pandas.columns.tolist())

Before cleaning: ['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'ratecodeid', 'store_and_fwd_flag', 'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge']
After cleaning: ['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'ratecodeid', 'store_and_fwd_flag', 'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge']


In [5]:
import yaml

schema = {
    "separator": ",",
    "columns": df_pandas.columns.tolist()
}

yaml_path = os.path.join(folder_path, "schema.yaml")

with open(yaml_path, "w") as f:
    yaml.dump(schema, f)

print("YAML schema saved at:", yaml_path)

YAML schema saved at: C:\After Backup\Virtual Internship\Week 6\2019\schema.yaml


In [6]:
with open(yaml_path, "r") as f:
    loaded_schema = yaml.safe_load(f)

expected_columns = loaded_schema["columns"]
separator = loaded_schema["separator"]

print("Expected Columns from YAML:", expected_columns)
print("Separator:", separator)

Expected Columns from YAML: ['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'ratecodeid', 'store_and_fwd_flag', 'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge']
Separator: ,


In [7]:
# Read just 5 rows to get column names
df_check = pd.read_csv(combined_file_path, sep=separator, nrows=5, on_bad_lines='skip')

actual_columns = df_check.columns.tolist()

# Check column match
if actual_columns == expected_columns:
    print("Column name and count match the YAML schema")
else:
    print("Mismatch!")
    print("From CSV:", actual_columns)
    print("From YAML:", expected_columns)

Column name and count match the YAML schema


In [8]:
import gzip

gz_output_path = os.path.join(folder_path, "combined_3_months_cleaned.txt.gz")

columns = expected_columns

first_chunk = True

chunk_size = 100000

reader = pd.read_csv(combined_file_path, chunksize=chunk_size, low_memory=False, on_bad_lines='skip')

with gzip.open(gz_output_path, 'wt', encoding='utf-8') as f_out:
    for chunk in reader:
        chunk.columns = chunk.columns.str.strip().str.replace('[^A-Za-z0-9_]+', '', regex=True)

        chunk.to_csv(f_out, sep='|', index=False, header=first_chunk)
        first_chunk = False

print("File successfully written to:", gz_output_path)

File successfully written to: C:\After Backup\Virtual Internship\Week 6\2019\combined_3_months_cleaned.txt.gz


In [10]:
import pandas as pd
import gzip
import os

sample_df = pd.read_csv(combined_file_path, nrows=100000, low_memory=False, on_bad_lines='skip')

sample_df.columns = sample_df.columns.str.strip().str.replace('[^A-Za-z0-9_]+', '', regex=True)

sample_output_path = os.path.join(folder_path, "sample_100k_cleaned.gz")

sample_df.to_csv(sample_output_path, sep="|", index=False, compression="gzip")

print("Sample .gz file created:", sample_output_path)


Sample .gz file created: C:\After Backup\Virtual Internship\Week 6\2019\sample_100k_cleaned.gz


In [9]:
row_count = 0
chunk_size = 100000

for chunk in pd.read_csv(gz_output_path, sep='|', chunksize=chunk_size, compression='gzip', on_bad_lines='skip'):
    row_count += len(chunk)

df_sample = pd.read_csv(gz_output_path, sep='|', nrows=5, compression='gzip', on_bad_lines='skip')
column_count = df_sample.shape[1]

file_size_mb = os.path.getsize(gz_output_path) / (1024 ** 2)

print("\nFINAL SUMMARY:")
print("Total Rows           :", row_count)
print("Total Columns        :", column_count)
print(f"File Size            : {file_size_mb:.2f} MB")
print("File Path            :", gz_output_path)


FINAL SUMMARY:
Total Rows           : 22519712
Total Columns        : 18
File Size            : 389.38 MB
File Path            : C:\After Backup\Virtual Internship\Week 6\2019\combined_3_months_cleaned.txt.gz
